<a href="https://colab.research.google.com/github/rashida048/Apache-Beam/blob/main/Composite__ransfrom_CoGroupBy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install --quiet apache_beam

In [2]:
mkdir -p data

In [3]:
ls

data/  sample_data/


In [4]:
import apache_beam as beam

In [6]:
def SplitRow(element):
  return element.split(',')

def filtering(record):
  return record[3] == "Accounts"

def filter_on_count(element):
  name, count = element
  if count > 30:
    return element

p1 = beam.Pipeline()

input_collection = (p1
                    |'Read from text file' >> beam.io.ReadFromText('data/dept_data.txt')
                    |'Split rows' >> beam.Map(SplitRow)
                    )

accounts_count = (
    input_collection
    | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
    | 'Pair each Accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " + record[1], 1))
    | 'Group and sum1' >> beam.CombinePerKey(sum)
    | 'Count filter Accounts ' >> beam.Filter(filter_on_count)
    | 'Write results for Accounts' >> beam.io.WriteToText('data/Accounts')
)

hr_count = (
    input_collection
    | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
    | 'Pair each HR employee with 1' >> beam.Map(lambda record: ("HR, " + record[1], 1))
    | 'Group and sum2' >> beam.CombinePerKey(sum)
    | 'Count Filter HR' >> beam.Filter(filter_on_count)
    | 'Write results for HR' >> beam.io.WriteToText('data/HR')
)

p1.run()


In [7]:
!head -n 20 'data/Accounts-00000-of-00001'

('Accounts, Marco', 31)
('Accounts, Rebekah', 31)
('Accounts, Itoe', 31)
('Accounts, Edouard', 31)
('Accounts, Kyle', 62)
('Accounts, Kumiko', 31)
('Accounts, Gaston', 31)


In [8]:
!head -n 20 'data/HR-00000-of-00001'

('HR, Beryl', 62)
('HR, Olga', 31)
('HR, Leslie', 31)
('HR, Mindy', 31)
('HR, Vicky', 31)
('HR, Richard', 31)
('HR, Kirk', 31)
('HR, Kaori', 31)
('HR, Oscar', 31)


In [ ]:
def format_output(element):
  name, count = element
  return ','.join((name.enode('ascii'), str(count), 'Regular Employee'))

In [12]:
def SplitRow(element):
  return element.split(',')

def format_output(element):
  name, count = element
  return ','.join((name,str(count), 'Regular Employee'))

def filter_on_count(element):
  name, count = element
  if count > 30:
    return element

p2 = beam.Pipeline()

input_collection = (p2
                    |'Read from text file' >> beam.io.ReadFromText('data/dept_data.txt')
                    |'Split rows' >> beam.Map(SplitRow)
                    )

accounts_count = (
    input_collection
    | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
    | 'Pair each Accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " + record[1], 1))
    | 'Group and sum1' >> beam.CombinePerKey(sum)
    | 'Count filter Accounts ' >> beam.Filter(filter_on_count)
    | 'Regular accounts employee' >> beam.Map(format_output)
    | 'Write results for Accounts' >> beam.io.WriteToText('data/Accounts1')
)

hr_count = (
    input_collection
    | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
    | 'Pair each HR employee with 1' >> beam.Map(lambda record: ("HR, " + record[1], 1))
    | 'Group and sum2' >> beam.CombinePerKey(sum)
    | 'Count Filter HR' >> beam.Filter(filter_on_count)
    | 'Regular HR Employee' >> beam.Map(format_output)
    | 'Write results for HR' >> beam.io.WriteToText('data/HR1')
)

p2.run()


In [14]:
!head -n 20 'data/Accounts1-00000-of-00001'

Accounts, Marco,31,Regular Employee
Accounts, Rebekah,31,Regular Employee
Accounts, Itoe,31,Regular Employee
Accounts, Edouard,31,Regular Employee
Accounts, Kyle,62,Regular Employee
Accounts, Kumiko,31,Regular Employee
Accounts, Gaston,31,Regular Employee


In [15]:
!head -n 20 'data/HR1-00000-of-00001'

HR, Beryl,62,Regular Employee
HR, Olga,31,Regular Employee
HR, Leslie,31,Regular Employee
HR, Mindy,31,Regular Employee
HR, Vicky,31,Regular Employee
HR, Richard,31,Regular Employee
HR, Kirk,31,Regular Employee
HR, Kaori,31,Regular Employee
HR, Oscar,31,Regular Employee


In [16]:
class MyTransform(beam.PTransform):
  def expand(self, input_coll):
    a = (
        input_coll
        | 'Group and sum1' >> beam.CombinePerKey(sum)
        | 'Count filter Accounts ' >> beam.Filter(filter_on_count)
        | 'Regular accounts employee' >> beam.Map(format_output)
    )
    return a



def SplitRow(element):
  return element.split(',')

def format_output(element):
  name, count = element
  return ','.join((name,str(count), 'Regular Employee'))

def filter_on_count(element):
  name, count = element
  if count > 30:
    return element

p3 = beam.Pipeline()

input_collection = (p3
                    |'Read from text file' >> beam.io.ReadFromText('data/dept_data.txt')
                    |'Split rows' >> beam.Map(SplitRow)
                    )

accounts_count = (
    input_collection
    | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
    | 'Pair each Accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " + record[1], 1))
    | 'Composite accounts' >> MyTransform()
    | 'Write results for Accounts' >> beam.io.WriteToText('data/Accounts2')
)

hr_count = (
    input_collection
    | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
    | 'Pair each HR employee with 1' >> beam.Map(lambda record: ("HR, " + record[1], 1))
    | 'Composite HR' >> MyTransform()
    | 'Write results for HR' >> beam.io.WriteToText('data/HR2')
)

p3.run()

In [17]:
!head -n 20 'data/Accounts2-00000-of-00001'

Accounts, Marco,31,Regular Employee
Accounts, Rebekah,31,Regular Employee
Accounts, Itoe,31,Regular Employee
Accounts, Edouard,31,Regular Employee
Accounts, Kyle,62,Regular Employee
Accounts, Kumiko,31,Regular Employee
Accounts, Gaston,31,Regular Employee


In [18]:
!head -n 20 'data/HR2-00000-of-00001'

HR, Beryl,62,Regular Employee
HR, Olga,31,Regular Employee
HR, Leslie,31,Regular Employee
HR, Mindy,31,Regular Employee
HR, Vicky,31,Regular Employee
HR, Richard,31,Regular Employee
HR, Kirk,31,Regular Employee
HR, Kaori,31,Regular Employee
HR, Oscar,31,Regular Employee


In [21]:
def retTuple(element):
  thisTuple=element.split(',')
  return (thisTuple[0], thisTuple[1:])

p4 = beam.Pipeline()

dep_rows = (p4
            | "Reading File 1" >> beam.io.ReadFromText("data/dept_data.txt")
            | "Pair each employee with key" >> beam.Map(retTuple))

loc_rows = (p4
            | "Reading File 2" >> beam.io.ReadFromText("data/location.txt")
            | "Pair each loc with key" >> beam.Map(retTuple)
            )

results = ({"dept_data": dep_rows, 'loc_data': loc_rows}
           |beam.CoGroupByKey()
           |'Write results' >> beam.io.WriteToText('data/results4'))

p4.run()

In [22]:
!head -n 20 "data/results4-00000-of-00001"

('149633CM', {'dept_data': [['Marco', '10', 'Accounts', '1-01-2019'], ['Marco', '10', 'Accounts', '2-01-2019'], ['Marco', '10', 'Accounts', '3-01-2019'], ['Marco', '10', 'Accounts', '4-01-2019'], ['Marco', '10', 'Accounts', '5-01-2019'], ['Marco', '10', 'Accounts', '6-01-2019'], ['Marco', '10', 'Accounts', '7-01-2019'], ['Marco', '10', 'Accounts', '8-01-2019'], ['Marco', '10', 'Accounts', '9-01-2019'], ['Marco', '10', 'Accounts', '10-01-2019'], ['Marco', '10', 'Accounts', '11-01-2019'], ['Marco', '10', 'Accounts', '12-01-2019'], ['Marco', '10', 'Accounts', '13-01-2019'], ['Marco', '10', 'Accounts', '14-01-2019'], ['Marco', '10', 'Accounts', '15-01-2019'], ['Marco', '10', 'Accounts', '16-01-2019'], ['Marco', '10', 'Accounts', '17-01-2019'], ['Marco', '10', 'Accounts', '18-01-2019'], ['Marco', '10', 'Accounts', '19-01-2019'], ['Marco', '10', 'Accounts', '20-01-2019'], ['Marco', '10', 'Accounts', '21-01-2019'], ['Marco', '10', 'Accounts', '22-01-2019'], ['Marco', '10', 'Accounts', '23-01-